In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import time
import os
import holidays
import pickle
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
df = pd.read_excel('./GEFCom2014-E.xlsx')
df['dow'] = df.Date.apply(lambda x: x.dayofweek)
df['month'] = df.Date.apply(lambda x: x.month)
df = df[df.load.isnull().sum():]
ush = holidays.US()
df['is_holiday'] = 1 * df.Date.apply(lambda x: x in ush)
df = df.reindex(columns=('month', 'dow', 'Hour', 'is_holiday', 'T', 'load', 'Date'))
offset = df.load.mean()
scale = df.load.std()
df.load -= df.load.mean()
df.load /= df.load.std()
df['T'] -= df['T'].mean()
df['T'] /= df['T'].std()
del df['Date']


df.month = np.cos(2*np.pi/12*df.month)
df.Hour = np.cos(2*np.pi/24*df.Hour)
df.dow = np.cos(2*np.pi/7*df.dow)

In [ ]:
train = df[:7*len(df)//8]
# val = df[3*len(df)//4:7*len(df)//8]
test = df[7*len(df)//8:]

# train = data[:int(0.7*len(data))]
# val = data[int(0.7*len(data)):int(0.9*len(data))]
# test = data[int(0.9*len(data)):]

nt = np.array(train)
# nv = np.array(val)
ntt = np.array(test)
x_train = nt[:,:-1]
y_train = nt[:,-1]
x_val = ntt[:,:-1]
y_val = ntt[:,-1]

In [ ]:
def row2seq_rnn(data, ys, history=24, future = 0):
    xl = []
    xff = []
    yl = []
    yf = []
    for i in np.arange(history+1, len(data)-max(future,1)):
        tmp1 = data[i-history:i+1]
        tmp2 = ys[i-history-1:i].reshape(-1,1)
        xl.append(np.append(tmp1, tmp2, axis=1))
        xff.append(data[i+1:i+future+1])
        yl.append(ys[i])
        yf.append(ys[i:i+future+1])
    return np.array(xl), np.array(xff), np.array(yl), np.array(yf)
def batch(*vars, size=512):
    for i in range(0, min(len(v) for v in vars), size):
        yield (v[i:i+size] for v in vars)

In [ ]:
xt, xtf, yt, ytf = row2seq_rnn(x_train, y_train, history=24, future=24)
# xv, xvf, yv, yvf = row2seq_rnn(x_val, y_val, history=48, future=24)
xtt, xttf, ytt, yttf = row2seq_rnn(np.array(test)[:,:-1], np.array(test)[:,-1], history=24, future=24)

In [ ]:
xt.shape, xtf.shape, yt.shape, ytf.shape

In [ ]:
bs = 1024*2
n_hidden = 32

In [ ]:
cond = lambda i, s, p: i < t_max
def body(i, s, preds):
    tmp = tf.concat((xf[:,i], preds[:,-1:]), axis=1)
    tmp.set_shape((None,xt.shape[-1]))
    out, state = lstm(tmp, s)
    new_pred = tf.layers.dense(out, 1, name='predict', reuse=True)
    return tf.add(i,1), state, tf.concat((preds, new_pred), axis=1)

In [ ]:
ew = lambda: tf.nn.rnn_cell.LSTMStateTuple(tf.TensorShape((None, n_hidden)), tf.TensorShape((None, n_hidden)))
g = tf.Graph()
with g.as_default():
    x = tf.placeholder(tf.float32, (None, None, xt.shape[2]), name='x')
    xf = tf.placeholder(tf.float32, (None, None, xtf.shape[2]), name='x_future')
    y = tf.placeholder(tf.float32, (None), name='y')
    yf = tf.placeholder(tf.float32, (None, None), name='y_future')
    regularization = tf.placeholder_with_default(0.0, (), name='reg')
    batch_size = tf.shape(x)[0]
    learning_rate = tf.placeholder_with_default(0.001, (), name='learning_rate')
    t_max = tf.shape(xf)[1]
    lstm = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.BasicLSTMCell(n_hidden) for _ in range(2)])
    
    outputs, state = tf.nn.dynamic_rnn(lstm, x, dtype=tf.float32)
    for ttt in state:
        for tt in ttt:
            tt.set_shape((None, n_hidden))
    i0 = tf.constant(0)
    out0 = tf.layers.dense(outputs[:,-1], 1, name='predict')
    
    _, _, predictions = tf.while_loop(cond, body, loop_vars=[i0, state, out0],
                        shape_invariants=[i0.get_shape(), 
                                          (ew(), ew()), tf.TensorShape([None, None])])
    
    single_pred = tf.layers.dense(outputs[:,-1], 1, name='predict', reuse=True)

#     reg_loss = tf.add_n([tf.nn.l2_loss(i) for i in tf.trainable_variables() if 'bias' not in i.name])
    err_loss = tf.reduce_mean((y[:,None]-single_pred)**2)
    loss = err_loss
#     loss = tf.reshape(reg_loss * regularization / tf.cast(batch_size, tf.float32), []) + err_loss
    
    err_losses = tf.reduce_mean((predictions-yf)**2)
#     many_loss = tf.reshape(reg_loss * 0.005 / tf.cast(batch_size, tf.float32), []) + err_losses
    many_loss = err_losses
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    bigger_train_op = tf.train.AdamOptimizer(learning_rate).minimize(many_loss)
    saver = tf.train.Saver()

In [ ]:
try:
    while True:
        tf.get_default_session().close()
except:
    pass
sess = tf.InteractiveSession(graph=g)
sess.run(tf.global_variables_initializer())

In [ ]:
def train_epoch(lr=0.001, batch_size=bs):
    perm = np.random.permutation(len(xt))
    errrrs = []
    for xs, ys, xfs, yfs in batch(xt[perm], yt[perm], xtf[perm], ytf[perm], size=batch_size):
        _, l = sess.run((bigger_train_op, many_loss), feed_dict={x:xs, y:ys, xf:xfs, yf:yfs, learning_rate:lr})
        errrrs.append(l)
    return errrrs

def evaluate(batch_size=bs):
    ls = []
    n = 0
    perm = np.random.permutation(len(xtt))
    for xs, ys, xfs, yfs in batch(xtt[perm], ytt[perm], xttf[perm], yttf[perm], size=batch_size):
        ls.append(sess.run(many_loss, feed_dict={x:xs, y:ys, xf:xfs, yf:yfs,})*len(xs))
        n += len(xs)
    return np.sum(ls)/n

In [ ]:
train_errs = []
test_errs = []
for i in range(len(test_errs), 300):
    start = time.time()
    train_err = train_epoch(batch_size=bs)
    train_errs.extend(train_err)
    test_errs.append(evaluate(bs*2))
    end = time.time()

    if i>1 and test_errs[-1]==min(test_errs):
        saver.save(sess, './results/rnn_2L48h_24p_24f/model', global_step=len(test_errs))
    print(f'Epoch {i} ({end-start:.2f}s): train_loss={train_errs[-1]:.4f}, test_loss={test_errs[-1]:.4f}')

In [ ]:
def predict(bs):
    l = []
    for xs, xfs, ys in batch(xtt, xttf,  ytt, size=bs):
        l.append(sess.run(predictions, feed_dict={x:xs, xf:xfs, y:ys}))
    return np.concatenate(l,0)

In [ ]:
pos = predict(bs*2)*scale + offset
yov = yttf*scale + offset
loss_by_horizon = ((yov-pos)**2).mean(0).astype(np.float32)
loss_by_horizon